- simplify and generalize game
- do not save on memory all games
- make it with pytorch and monitor with wandb
- Build a solver for the game to evaluate

In [1]:
# te

ModuleNotFoundError: No module named 'src.zero'

In [8]:
import random
from sklearn.neural_network import MLPClassifier
import sklearn
from copy import deepcopy
import numpy as np
import pandas as pd
class Game:
    def __init__(self,):
        self.state = [0,0,0,0]
        self.action_space = [0,1,2,3]
        self.player = -1
    def reset(self,):
        self.state = [0,0,0,0]
        self.action_space = [0,1,2,3]
        self.player = -1
        return self
    def play(self, action):
        """Reward: 0 (no outcome), 1 (player win), -1 player lose or illegal"""
        self.player *=-1
        if self.state[action]!=0:
            reward = -1
        else:
        
            
            self.state[action] = self.player
            # specific
            if abs(sum(self.state[0:2]))==2*self.player or abs(sum(self.state[1:3]))==2*self.player or abs(sum(self.state[2:4]))==2*self.player:
                reward = self.player
            else:
                reward = 0
        
        
        
        
        return self.state.copy(), reward, self.player
    
class Buffer:
    """History of plays. Doing the preprocessing here?"""
    def __init__(self,):
        self.all_time_buffer = []
        self.current_buffer = []
    def add_to_buffer(self, state, reward, player, action, game):
        buffer_entry = (state, reward, player, action, game)
        self.all_time_buffer.append(buffer_entry)
        
    def processed_buffer(self, player):
        """For now we consider only winning states for player"""
        buffer_df = pd.DataFrame(self.all_time_buffer, columns=["state", "reward", "player", "action", "n_game"])
        # taking only the actions of the games were player 1 is winning
        mask_winning = (buffer_df.reward==1) & (buffer_df.player==player)
        n_games_win = buffer_df[mask_winning].n_game.unique()
        winning_buffer_df = buffer_df[(buffer_df.n_game.isin(n_games_win)) & (buffer_df.player==player)]
        
        winning_states = np.array([state for state in winning_buffer_df.state])
        winning_actions = np.array([[action] for action in winning_buffer_df.action])
        return winning_states, winning_actions
        #self.current_buffer.append(buffer_entry)
        
class Model:
    def __init__(self, action_space, state):
        self.action_space = action_space
        self.state = state
        self.model = MLPClassifier(hidden_layer_sizes=(10,10))
        self.is_fitted = False
        
    def train(self, states, actions):
        #try:
        self.model.fit(X=states, y=actions)
        #except:
        #    self.model.fit(X=states.reshape(1,-1), y=actions)
            
        self.is_fitted = True
        
    
    def predict_action(self, state):
        if self.is_fitted:
            return self.model.predict(state.reshape(1,-1))[0]
        else:
            return random.choice(self.action_space)
class Simulation:
    """Simulation based on n games, player 1, player 2, ratio of model pred vs random and output processed buffer"""
    def simulate(self,model1, model2, n_games, ratio_random, game):
        buffer = Buffer()
        
        for n in range(n_games):
            game = game.reset()
            state = game.state.copy()
            while True:
                action = model1.predict_action(np.array(state))
                state, reward, player = game.play(action=action)
                buffer.add_to_buffer(state, reward, player, action, n)
                if reward !=0:
                    break
                action_2 = model2.predict_action(np.array(state))
                state, reward_2, player_2 = game.play(action=action_2)
                buffer.add_to_buffer(state, reward_2, player_2, action_2, n)
                if reward_2 !=0:
                    break
            
        return buffer
        
class Zero:
    """Model 1 vs model 2"""
    def __init__(self,):
        self.model_ini = Model(action_space=[0,1,2,3], state=[0,0,0,0])
    
    
    def run_once(self, model1, model2):
        simulation = Simulation()
        buffer = simulation.simulate(model1=model1, model2=model2, n_games=1000, ratio_random=0, game=Game())
        return buffer
    
    def run(self):
        # main model is model 1
        main_model = deepcopy(self.model_ini)
        adv_model = deepcopy(self.model_ini)
        for n in range(5):
            print("n", n)
            if n%2==0:
                print("for model 1")
                buffer = self.run_once(main_model, adv_model)
                player=1
            else:
                print("for model 2")
                buffer = self.run_once(adv_model, main_model)
                player = -1

            X, y = buffer.processed_buffer(player=player) 
            if X.size==0:
                return buffer
                break
            print("training")
            avg_model = deepcopy(main_model)
            main_model.train(X, y)
                
    
        
        

In [9]:
np.array([]).size

0

In [10]:
zero = Zero()

In [11]:
buffer = zero.run()

n 0
for model 1
training
n 1
for model 2


/Users/pablocanadapereira/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/pablocanadapereira/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [12]:
buffer.all_time_buffer

[([1, 0, 0, 0], 0, 1, 0, 0),
 ([1, 0, 0, 0], -1, -1, 0, 0),
 ([1, 0, 0, 0], 0, 1, 0, 1),
 ([1, 0, 0, 0], -1, -1, 0, 1),
 ([1, 0, 0, 0], 0, 1, 0, 2),
 ([1, 0, 0, 0], -1, -1, 0, 2),
 ([1, 0, 0, 0], 0, 1, 0, 3),
 ([1, 0, 0, 0], -1, -1, 0, 3),
 ([0, 1, 0, 0], 0, 1, 1, 4),
 ([0, 1, 0, 0], -1, -1, 1, 4),
 ([0, 1, 0, 0], 0, 1, 1, 5),
 ([0, 1, 0, 0], -1, -1, 1, 5),
 ([1, 0, 0, 0], 0, 1, 0, 6),
 ([1, 0, 0, 0], -1, -1, 0, 6),
 ([0, 1, 0, 0], 0, 1, 1, 7),
 ([0, 1, 0, 0], -1, -1, 1, 7),
 ([0, 0, 1, 0], 0, 1, 2, 8),
 ([0, 0, 1, 0], -1, -1, 2, 8),
 ([0, 1, 0, 0], 0, 1, 1, 9),
 ([0, 1, 0, 0], -1, -1, 1, 9),
 ([0, 0, 0, 1], 0, 1, 3, 10),
 ([0, 0, 0, 1], -1, -1, 3, 10),
 ([0, 1, 0, 0], 0, 1, 1, 11),
 ([0, 1, 0, 0], -1, -1, 1, 11),
 ([0, 0, 1, 0], 0, 1, 2, 12),
 ([0, 0, 1, 0], -1, -1, 2, 12),
 ([0, 0, 1, 0], 0, 1, 2, 13),
 ([0, 0, 1, 0], -1, -1, 2, 13),
 ([1, 0, 0, 0], 0, 1, 0, 14),
 ([1, 0, 0, 0], -1, -1, 0, 14),
 ([0, 1, 0, 0], 0, 1, 1, 15),
 ([0, 1, 0, 0], -1, -1, 1, 15),
 ([1, 0, 0, 0], 0, 1, 0, 16)

In [277]:
[(player, n_game) for (state, reward, player, action, n_game) in buffer.all_time_buffer if reward==1]

[(1, 2)]

In [181]:
buffer.all_time_buffer[]

[([1, 0, 0, 0], 0, 1, 0, 0),
 ([1, -1, 0, 0], 0, -1, 1, 0),
 ([1, -1, 1, 0], 0, 1, 2, 0),
 ([1, -1, 1, 0], -1, -1, 2, 0),
 ([1, 0, 0, 0], 0, 1, 0, 1),
 ([1, -1, 0, 0], 0, -1, 1, 1),
 ([1, -1, 1, 0], 0, 1, 2, 1),
 ([1, -1, 1, 0], -1, -1, 0, 1),
 ([0, 0, 0, 1], 0, 1, 3, 2),
 ([0, -1, 0, 1], 0, -1, 1, 2),
 ([0, -1, 1, 1], 1, 1, 2, 2)]

In [ ]:
states = [state for state, re in buffer.all_time_buffer if ]

In [176]:
buffer.all_time_buffer

[([1, 0, 0, 0], 0, 1, 0, 0),
 ([1, -1, 0, 0], 0, -1, 1, 0),
 ([1, -1, 1, 0], 0, 1, 2, 0),
 ([1, -1, 1, 0], -1, -1, 2, 0),
 ([1, 0, 0, 0], 0, 1, 0, 1),
 ([1, -1, 0, 0], 0, -1, 1, 1),
 ([1, -1, 1, 0], 0, 1, 2, 1),
 ([1, -1, 1, 0], -1, -1, 0, 1),
 ([0, 0, 0, 1], 0, 1, 3, 2),
 ([0, -1, 0, 1], 0, -1, 1, 2),
 ([0, -1, 1, 1], 1, 1, 2, 2)]

In [206]:
import pandas as pd
import numpy as np
buffer_df = pd.DataFrame(buffer.all_time_buffer, columns=["state", "reward", "player", "action", "n_game"])

In [198]:
mask_winning = (buffer_df.reward==1) & (buffer_df.player==1)
n_games_win = buffer_df[mask_winning].n_game.unique()
winning_buffer_df = buffer_df[(buffer_df.n_game.isin(n_games_win)) & (buffer_df.player==1)]

In [208]:
winning_states = np.array([state for state in winning_buffer_df.state])
winning_actions = [action for action in winning_buffer_df.action]

In [209]:
winning_actions

[3, 2]

In [ ]:
class TicTacToe():
    def __init__(self):
        self.board = [0,0,0,0,0,0,0,0,0]  
        self.action_space=[0,1,2,3,4,5,6,7,8]
        self.player=1
        self.reward_win=1
        self.reward_draw = 0.5
        self.list_index_win = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6]]
    def reset(self,):
        self.board = [0,0,0,0,0,0,0,0,0]  
        self.player = 1

    def _get_possible_next_states(self, state):
        """Give possible states from the state position. Action is the same as index. Potentially problematic.
        We are passing current state as argument, not necessarily the state of the board as it is."""
        possible_actions = self._get_possible_actions(state=state)
        possible_next_states = []
        
        for action in possible_actions:
            board = state.copy()
            board[action]=self.player
            possible_next_states.append(board)
            
        return possible_next_states

        
    def _get_possible_actions(self, state): 
        """Return all posible actions in the form of array [1,2] for example for [1,0,0,-1]"""
        return np.where(np.array(state)==0)[0]
    
        
    def render_table(self,):
        print(np.array(self.board).reshape(3,3), "Next player:", self.player)

    def _is_legal(self, action, state=None):
        """We allow to ways of calling the method. Directly to check if an state is win or lost or by chcking the board"""
        is_legal=False
        
        if state is None:
            state = self.board.copy()

        if state[action]==0:
            is_legal=True
        return is_legal

    def _is_win(self, state=None):
        """We allow to ways of calling the method. Directly to check if an state is win or lost or by chcking the board"""
        is_win = False
        if state is None:
            state = self.board.copy()
        state_array=np.array(state)
        for win_combination in self.list_index_win:
            if abs(sum(state_array[win_combination]))==3:
                is_win = True
                return is_win


    def play(self, action):
        """From the perspective of player 1"""
        output = None
        if action not in self.action_space:
            raise ValueError(f"Action: {action} is not allowed from the action space: {self.action_space}")
            
        if self._is_legal(action):
            board_temp = self.board.copy()
            board_temp[action]= self.player
            self.board = board_temp.copy()
            is_win=self._is_win()
            if is_win: 
                output = self.reward_win

            elif not is_win and len(self._get_possible_actions(self.board))==0:
                output = self.reward_draw
            
            self.player = self.player*-1
            
        else:
            raise ValueError("Illegal move")
        return output